In [68]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import utils
import sklearn as skl
import sklearn.preprocessing

## Get genre data and track-ID:s

In [18]:
# Load data
tracks = utils.load("data/fma_metadata/tracks.csv")

In [19]:
# Get indexes
small_indexes = tracks['set', 'subset'] == 'small'
train_indexes = tracks['set', 'split'] == 'training'
val_indexes   = tracks['set', 'split'] == 'validation'
test_indexes  = tracks['set', 'split'] == 'test'

In [173]:
# Remove erronous files
train_indexes[133297] = False
train_indexes[99134] = False
train_indexes[108925] = False

In [174]:
# Get genre labels
y_train = tracks.loc[small_indexes & train_indexes, ('track', 'genre_top')]
y_val = tracks.loc[small_indexes & val_indexes, ('track', 'genre_top')]
y_test = tracks.loc[small_indexes & test_indexes, ('track', 'genre_top')]

In [175]:
# Encode genre labels

enc = skl.preprocessing.LabelEncoder()
y_train_encoded = enc.fit_transform(y_train)
y_val_encoded = enc.transform(y_val)
y_test_encoded = enc.transform(y_test)

# See encoding
labels = np.unique(y_train)
print(labels)
print(enc.transform(labels))

['Electronic' 'Experimental' 'Folk' 'Hip-Hop' 'Instrumental'
 'International' 'Pop' 'Rock']
[0 1 2 3 4 5 6 7]


In [176]:
# Save genre labels
np.savez("data/processed/train_y", y_train_encoded)
np.savez("data/processed/val_y", y_val_encoded)
np.savez("data/processed/test_y", y_test_encoded)

In [177]:
# Get track-ids
id_train = np.array(tracks.loc[small_indexes & train_indexes, ["track_id"]].index)
id_val   = np.array(tracks.loc[small_indexes & val_indexes, ["track_id"]].index)
id_test  = np.array(tracks.loc[small_indexes & test_indexes, ["track_id"]].index)

## Preprocess music data

In [167]:
## Constants
AUDIO_DIR = "data/fma_small"
SAMPLING_RATE = 12000
MEL_BINS = 96
HOP_SIZE = 256
MAX_LENGTH = 1405

In [163]:
padded = []

def process_audio(track_IDs, verbose = False):

    N = len(track_IDs)
    out = np.zeros((N, MEL_BINS, MAX_LENGTH))

    for index, track_id in enumerate(track_IDs):
        # Print progress
        if verbose:
            print("At {}/{}".format(index + 1, N))

        # Get path to filename
        tmp = "{:06d}".format(track_id)
        path = AUDIO_DIR + "/"  + tmp[0:3] + "/" + tmp + ".mp3"

        # Load data
        audio, sample_rate = librosa.core.load(path, sr = SAMPLING_RATE)

        # Convert to log-mel-spectogram
        mel_spectogram = librosa.feature.melspectrogram(y = audio, 
                                                       sr = SAMPLING_RATE,
                                                      hop_length = HOP_SIZE,
                                                       n_mels = MEL_BINS)
        log_mel_spectogram = librosa.power_to_db(mel_spectogram, ref = np.max)


        # Make sure data is always same length (sometimes 1406 or 1407 due to rounding)
        # Some samples are super short, we pad these with zeros in the end
        try:
            out[index, :, :] = log_mel_spectogram[:,:MAX_LENGTH]
        except:
            out[index, :, :log_mel_spectogram.shape[1]] = log_mel_spectogram
            padded.append(index)
        
    return out

In [181]:
# Save test data
out = process_audio(id_test, verbose = True)
np.savez("data/processed/test_X", out)

At 1/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 2/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 3/800


...

/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 799/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 800/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [182]:
# Save validation data
out = process_audio(id_val, verbose = True)
np.savez("data/processed/val_X", out)

At 1/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 2/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 3/800


...

/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 799/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 800/800


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [179]:
# Save training data
padded = []
out = process_audio(id_train, verbose = True)
np.savez("data/processed/train_X", out)
print(padded)

At 1/6397


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 2/6397


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 3/6397


...

At 6396/6397


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


At 6397/6397


/Users/toomasliiv/.local/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


[3495, 3496, 3497]


In [180]:
out.shape

(6397, 96, 1405)